#### CSC 180 Intelligent Systems 

#### William Lorence

#### California State University, Sacramento


# Project 2: Modern Low Footprint Cyber Attack Protection

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

path = "./dataset/"
save_path = "./models/"

training_set = os.path.join(path,"UNSW_NB15_training-set.csv")
test_set = os.path.join(path,"UNSW_NB15_test-set.csv")

df_training_set = pd.read_csv(training_set)
df_test_set = pd.read_csv(test_set)

print(df_training_set[['attack_cat']].to_string)

<bound method DataFrame.to_string of        attack_cat
0          Normal
1          Normal
2          Normal
3          Normal
4          Normal
...           ...
175336    Generic
175337  Shellcode
175338    Generic
175339    Generic
175340    Generic

[175341 rows x 1 columns]>
